# Detection of phishing websites using ML

## 1.Introduction to Notebook

This notebook was created as a part of a workshop in computational learning ,  a course me and my partner were partaking in Afeka College of Engineering . 

The final assignment of this workshop is utilizing machine learning in a certain goal.

Me, Niv Arad , and my partner, Omri Chen Yosef, were assigned to use ML in order to detect Phishing scams, as the open source dataset found on the web is not being updated fast enough and there is a need to classify benign website and spams quickly and reliably .




## 2. Objectives

A phishing site often employs social engineering tactics by imitating legitimate URLs and web pages. This project aims to develop machine learning and deep learning models using a specially compiled dataset for identifying phishing sites. The dataset includes both malicious phishing URLs and safe website URLs, from which necessary features based on URL and website content are derived. At the end we were trying a few different models , and compared them .

## 3. Data collections

The first step in this project was to find datasets that can serve our model. 

In order to train our model we had to find to find dataset of both legitimate urls and phishing too.

The dataset of phishing url can be found from the opensource service called PhishTank.This dataset is getting updated daily. To download the data: **https://www.phishtank.com/developer_info.php**

For the legitimate URLs, we found a source that has a collection of benign and phishingURLs. The source of the dataset is Aalto University based in Finland, **https://research.aalto.fi/en/datasets/phishstorm-phishing-legitimate-url-dataset**. 
The following dataset has the collections mentioned above merged , so we will need to make an extra step and drop all the rows of the urls who are not benign.

** We decided not to use only the dataset obtained from the aalto universitt altough has benign and phishing urls , because it's not updated regulary as the phishtank url dataset

## 4. Collection of URL dataset

### 4.1. Phishing urls dataset

In [ ]:
import pandas as pd

In [ ]:
phish_data = pd.read_csv("/kaggle/input/websites-datasets/verified_online.csv")

In [ ]:
phish_data.head()

In [ ]:
phish_data.tail()

In [ ]:
phish_data.shape

In [ ]:
#Collecting 1000 Phishing URLs randomly
phish_url = phish_data.sample(n = 40000, random_state = 12).copy()
phish_url = phish_url.reset_index(drop=True)
phish_url.head()

In [ ]:
phish_url.shape

### 3.2. Legitimate urls dataset


we are loading only the first column because there is a problem loading all the file

In [ ]:
legit_data = pd.read_csv("/kaggle/input/websites-datasets/urlset.csv", encoding="ISO-8859-1", usecols=[0])

In [ ]:
legit_data.head()

In [ ]:
legit_data.shape

In [ ]:
#Collecting 1000 Legitimate URLs randomly
legi_url = legit_data.sample(n = 40000, random_state = 12).copy()
legi_url = legi_url.reset_index(drop=True)
legi_url.head()

In [ ]:
legi_url.shape

## 5. Feature Extraction


### 5.1. URL-based features

These will be the features we will extract from the url , we gathered those from the artical that was presented in class, more explaination will be following for every feature

- domain of url
- IP address in URL
- length of url
- depth of url
- "http/https" in Domain
- Using URL Shortening Services “TinyURL”
- Count of prefix or sufix "-" in Domain
- Count of prefix or sufix "_" in Domain
- Sub-domain length
- "client" in string
- "admin" in string
- "login" in string
- "server" in string

In [ ]:
# importing required packages for this section
from urllib.parse import urlparse,urlencode
import ipaddress
import re

#### 5.1.1. Domain of url

We are just extracting the domain present in the URL. This feature doesn't have much significance in the training , because we the domain name alone can't give any indication of the type of website it is.

In [ ]:
def getDomain(url):
    domain = urlparse(url).netloc
    if re.match(r"^www.",domain):
        domain = domain.replace("www.","")
    return domain

#### 5.1.2. Length of URL

Phishing URLs may have excessively long domains or domain strings that appear unusual or unrelated to the legitimate domain.Longer domain names indicates phishing scams.

In [ ]:
def lengthURL(url):
    return len(url)

#### 5.1.3. Depth of URL

Computes the depth of the URL. This feature calculates the number of sub pages in the given url based on the '/'.

The value of feature is a numerical based on the URL.

In [ ]:
def depthURL(url):
    s = urlparse(url).path.split('/')
    depth = 0
    for j in range(len(s)):
        if len(s[j])!=0:
            depth = depth+1
    return depth

#### 5.1.4. "http/https" in Domain name

Checks for the presence of "http/https" in the domain part of the URL.The phishers add numerous times the "https" to the url in order to trick it's users and the make them feel like the website is legitimate as the https needs the ssl certificate which ensured the website owner identity.

In [ ]:
def httpDomain(url):
    if "https" in url:
        return 1
    else:
        return 0

#### 5.1.5. URL shortening

URL shortening is a method on the “World Wide Web” in which a URL may be made considerably smaller in length and still lead to the required webpage. This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL.

If the URL is using Shortening Services, the value assigned to this feature is 1 or else 0.

In [ ]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [ ]:
def tinyURL(url):
    match = re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

#### 5.1.6. Preffix or Suffix "-" in Domain

Phishing websites are crafted to mislead individuals into thinking they are on official sites, often mimicking the appearance of famous brands or institutions. A frequent trick used in these malicious domains is the strategic placement of a hyphen ("-") to fool users.

This tactic involves adding a hyphen to create a domain name that closely resembles the genuine domain it aims to mimic. For instance, instead of the authentic "website.com," a phishing site might use "web-site.com" or "websi-te.com." The goal behind using the hyphen is to take advantage of users who are not paying close attention or to capitalize on typosquatting, where users accidentally enter incorrect characters or misspell a real domain name.

In [ ]:
def hyphenURL(url):
    if '-' in urlparse(url).netloc:
        return 1
    else:
        return 0

#### 5.1.7. Length of Sub-domain

Phishing URLs may have excessively long subdomains or subdomain strings that appear unusual or unrelated to the legitimate domain. For example, "login.yourbank.com.phishingsite.com."

In [ ]:
def subURl(url):
    p = urlparse(url).path.split('.')
    sub = 0
    for j in range(len(p)):
        if len(p[j])!=0:
            sub = sub+1
    return sub

#### Keywords in URL

Look for keywords in the URL that suggest fraudulent activities, such as "login", "client", "admin", "server", or "account." Phishing URLs often try to deceive users into believing they are accessing sensitive pages.

#### 5.1.8. search for keywords in url

In [ ]:
def keyWordsURL(url):
    keywords = ["client","login","admin","server","account"]
    if any(word in url for word in keywords):
        return 1
    else:
        return 0

### 5.2. Domain-based features

- DNS record
- Age of domain
- End period of domain

In [ ]:
pip install python-whois

In [ ]:

# importing required packages for this section
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
from datetime import datetime

#### 4.2.1. DNS record

For phishing websites, either the claimed identity is not recognized by the WHOIS database or no records founded for the hostname. If the DNS record is empty or not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [ ]:
# DNS Record availability (DNS_Record)
# obtained in the featureExtraction function itself

#### 4.2.2. Age of domain

This feature can be extracted from WHOIS database. Most phishing websites live for a short period of time. The minimum age of the legitimate domain is considered to be 12 months for this project. Age here is nothing but different between creation and expiration time.

If age of domain > 12 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [ ]:
import whois
from datetime import datetime

def domainAge(domain_name):
    creation_date = domain_name.creation_date
    expiration_date = domain_name.expiration_date
    if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
        try:
            creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
            expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
        except:
            return 1
    if ((expiration_date is None) or (creation_date is None)):
        return 1
    elif ((type(expiration_date) is list) or (type(creation_date) is list)):
        return 1
    else:
        ageofdomain = abs((expiration_date - creation_date).days)
        if ((ageofdomain/30) < 6):
            age = 1
        else:
            age = 0
    return age

#### 4.2.3. End period of Domain

This feature can be extracted from WHOIS database. For this feature, the remaining domain time is calculated by finding the different between expiration time & current time. The end period considered for the legitimate domain is 6 months or less for this project.

If end period of domain > 6 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [ ]:
def domainEnd(domain_name):
    expiration_date = domain_name.expiration_date
    if isinstance(expiration_date,str):
        try:
            expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
        except:
            return 1
    if (expiration_date is None):
        return 1
    elif (type(expiration_date) is list):
        return 1
    else:
        today = datetime.now()
        end = abs((expiration_date - today).days)
        if ((end/30) < 6):
            end = 0
        else:
            end = 1
    return end

### 4.3. Content-based Features

- IFrame redirection
- Webiste forwarding

#### 4.3.1. IFrame redirection

IFrame is an HTML tag used to display an additional webpage into one that is currently shown. Phishers can make use of the “iframe” tag and make it invisible i.e. without frame borders. In this regard, phishers make use of the “frameBorder” attribute which causes the browser to render a visual delineation.

If the iframe is empty or repsonse is not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [ ]:
import requests
import re

In [ ]:
def iframe(response):
    if response == "":
        return 1
    else:
        if re.findall(r"[|]", response.text):
            return 0
        else:
            return 1

#### 4.3.2. Website forwarding

The fine line that distinguishes phishing websites from legitimate ones is how many times a website has been redirected. In our dataset, we find that legitimate websites have been redirected one time max. On the other hand, phishing websites containing this feature have been redirected at least 4 times.

In [ ]:
def forwarding(response):
    if response == "":
        return 1
    else:
        if len(response.history) <= 2:
            return 0
        else:
            return 1

## 5. URL feature extraction

Create a list and a function that calls the other functions and stores all the features of the URL in the list. We will extract the features of each URL and append to this list.

In [ ]:
import time
def featureExtraction(url, label):
    features = []
    
    start = time.time()
    #URL-based feature
    features.append(getDomain(url))
    features.append(lengthURL(url))
    features.append(depthURL(url))
    features.append(httpDomain(url))
    features.append(tinyURL(url))
    features.append(hyphenURL(url))
    features.append(subURl(url))
    features.append(keyWordsURL(url))
    end =time.time()
    parsing = end-start
    
    features.append(label)

    return (features,None)
    

### 5.1. Legitimate URLs:

Feature extraction is done on legitimate URLs.

In [ ]:
legi_url.head()

In [ ]:
#Extracting the features & storing them in a list
legi_features = []
label = 0
sum_parsing =0
sum_domain = 0
sum_content = 0

for i in range(0, 40000):
    url = legi_url["domain"][i]
    try:
        res = featureExtraction(url,label)
        legi_features.append(res[0])
    except:
        print("there was a problem with line ",i ,"the url is: ",url)
    # print(legi_features)

print("Parsing : ",sum_parsing,"\nDomain :",sum_domain,"\nContent :",sum_content)

In [ ]:
#Converting the list to dataframe
feature_name = ["domain","url_len","url_dep","httpDomain","tinyURL","hyphenURL",
                "subURl","keyWordURL","label"]

legitimate = pd.DataFrame(legi_features, columns=feature_name)

In [ ]:
legitimate = legitimate.drop(["domain"], axis=1)
legitimate.head()

In [ ]:
legitimate.shape

In [ ]:
legitimate.to_csv('/kaggle/working/legitimate.csv')

### 5.2. Phishing URLs:

exctractig the features of the phishing urls took a lot of time, and also there are a few lines in the dataset which has a problem to be computed ,so i did this on my local machine and now i am adding the file to the datasets, its called "phishing_finished"



### 5.2.1 Phishing URLs:
If you choose to use the feature extraction in that mentioned before and it worked please continue this section:

Feature extraction is done on phishing URLs.

In [ ]:
phish_url.drop(phish_url.columns.difference(['url']), 1, inplace=True)
phish_url.head()

exctractig the features of the phishing urls took a lot of time, and also there are a few lines in the dataset which has a problem to be computed ,so i did this on my local machine and now i am adding the file to the datasets, its called "phishing_finished"

In [ ]:

# #Extracting the features & storing them in a list
phish_features = []
label = 1

sum_parsing =0
sum_domain = 0
sum_content = 0
for i in range(0, 40000):
    url = phish_url["url"][i]
    res = featureExtraction(url,label)
    phish_features.append(res[0])

        
#     # print(legi_features)
print("finished extracting features from phishing URLS")

### 5.2.1 Phishing URLs:
If you choose to use the feature extraction in that mentioned before and it worked please continue this section:

Feature extraction is done on phishing URLs.

In [ ]:
# phish_features

In [ ]:
#Converting the list to dataframe
feature_name = ["domain","url_len","url_dep","httpDomain","tinyURL","hyphenURL",
                "subURl","keyWordURL","label"]

phishing = pd.DataFrame(phish_features, columns=feature_name)
phishing.head()

In [ ]:
phishing.tail()

In [ ]:
# phishing = phishing.drop(["domain"], axis=1)
phishing.to_csv('/kaggle/working/urldata.csv')

In [ ]:
phishing.shape

### 5.2.2 Phishing URLs:
If you chose not to use the feature extraction mentioned above or it not worked and got stuck  , you will need to load the data from the file we customly made in our local machine



In [ ]:
# phishing = pd.read_csv("/kaggle/input/phishing-computed/final_phishing.csv")

In [ ]:
# phishing.head()

In [ ]:
# phishing.tail()

### 5.3. Final dataset

In the above section we formed two dataframes of legitimate & phishing URL features. Now, we will combine them to a single dataframe and export the data to csv file for the Machine Learning training done in other notebook

In [ ]:
#Concatenating the dataframes into one 
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata = urldata.sample(frac=1, random_state=1).reset_index(drop=True)
urldata.head()

In [ ]:
urldata.tail()

In [ ]:
urldata.to_csv('/kaggle/working/urldata.csv', index=False)

## 6. Loading final datastet

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
urldata = pd.read_csv("/kaggle/working/urldata.csv")

In [ ]:
urldata = urldata.drop(["domain"], axis=1)

In [ ]:
urldata.head()

In [ ]:
urldata.tail()

## 7. Data Preprocessing and EDA of the URL dataset

### 7.1. Shape of the dataet

In [ ]:
urldata.shape

### 7.2. Dataset Columns

In [ ]:
urldata.columns

### 7.3. Dataset information

In [ ]:
urldata.info()

### 7.4. Dataset Summary

In [ ]:
urldata.describe

The above obtained result shows that the most of the data is made of 0's & 1's except 'Domain' & 'URL_Depth' columns. The Domain column doesnt have any significance to the machine learning model training. So dropping the 'Domain' column from the dataset.

### 7.5.Check of Null values

In [ ]:
urldata.isnull().sum()

In the feature extraction file, the extracted features of legitmate & phishing url datasets are just concatenated without any shuffling. This resulted in top 1000 rows of legitimate url data & bottom 1000 of phishing url data.

In [ ]:
urldata.columns

### 7.7. Max and Min of "url_len"

In [ ]:
print("Maximum of url length::",urldata["url_len"].max())
print("Minimum of url length::",urldata["url_len"].min())

### 7.8. Max and Min of "url_depth"

In [ ]:
print("Maximum of url depth::",urldata["url_dep"].max())
print("Minimum of url length::",urldata["url_dep"].min())

## 8. Data Visualization 

### 8.1. Plotting the data distribution

In [ ]:
urldata.hist(bins = 50,figsize = (15,15))
plt.show()

### 8.2. Correlation heatmap

In [ ]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(15,13))
# sns.heatmap(urldata.corr())
# plt.show()

plt.figure(figsize=(15,13))
cor = np.abs(urldata.corr())
sns.heatmap(cor, annot=True,cmap=plt.cm.Greens, vmin=-1, vmax=1)
plt.show()

## 8.3 Regplot

## 9. Splitting the Dataset

In [ ]:
y = urldata['label']
X = urldata.drop('label',axis=1)
X.shape, y.shape

In [ ]:
# Splitting the dataset into train and test sets: 70-30 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.4, random_state = 12)
X_train.shape, X_test.shape

## 10. Machine Learning models and Training

This data set comes under classification problem, as the input URL is classified as phishing (1) or legitimate (0). The supervised machine learning models (classification) considered to train the dataset:

- Decision Tree
- Random Forest
- XGBoost
- Logistic Regression
- Support Vectore Machines (SVM)
- Multilayer perceptrons
- Autoencoder Neural Network

In [ ]:
#importing packages
from sklearn.metrics import accuracy_score

In [ ]:
# Creating holders to store the model performance results
ML_Model = []
acc_train = []
acc_test = []

#function to call for storing the results
def storeResults(model, a,b):
    ML_Model.append(model)
    acc_train.append(round(a, 3))
    acc_test.append(round(b, 3))

### 10.1. Decision Tree Classifier

Decision trees are widely used models for classification and regression tasks. Essentially, they learn a hierarchy of if/else questions, leading to a decision. Learning a decision tree means learning the sequence of if/else questions that gets us to the true answer most quickly.

In the machine learning setting, these questions are called tests (not to be confused with the test set, which is the data we use to test to see how generalizable our model is). To build a tree, the algorithm searches over all possible tests and finds the one that is most informative about the target variable.

In [ ]:
# Decision Tree model 
from sklearn.tree import DecisionTreeClassifier

# instantiate the model 
tree = DecisionTreeClassifier(max_depth = 5)
# fit the model 
tree.fit(X_train, y_train)

In [ ]:
#predicting the target value from the model for the samples
y_test_tree = tree.predict(X_test)
y_train_tree = tree.predict(X_train)

#### Performance Evaluation

In [ ]:
#computing the accuracy of the model performance
acc_train_tree = accuracy_score(y_train,y_train_tree)
acc_test_tree = accuracy_score(y_test,y_test_tree)

print("Decision Tree: Accuracy on training Data: {:.3f}".format(acc_train_tree))
print("Decision Tree: Accuracy on test Data: {:.3f}".format(acc_test_tree))

In [ ]:

#checking the feature improtance in the model
plt.figure(figsize=(9,7))
n_features = X_train.shape[1]
plt.barh(range(n_features), tree.feature_importances_, align='center')
plt.yticks(np.arange(n_features), X_train.columns)
plt.xlabel("Feature importance")
plt.ylabel("Feature")
plt.show()

#### Storing Results:

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('Decision Tree', acc_train_tree, acc_test_tree)

### 10.2. Random Forest

Random forests for regression and classification are currently among the most widely used machine learning methods.A random forest is essentially a collection of decision trees, where each tree is slightly different from the others. The idea behind random forests is that each tree might do a relatively good job of predicting, but will likely overfit on part of the data.

If we build many trees, all of which work well and overfit in different ways, we can reduce the amount of overfitting by averaging their results. To build a random forest model, you need to decide on the number of trees to build (the n_estimators parameter of RandomForestRegressor or RandomForestClassifier). They are very powerful, often work well without heavy tuning of the parameters, and don’t require scaling of the data.

In [ ]:
# Random Forest model
from sklearn.ensemble import RandomForestClassifier

# instantiate the model
forest = RandomForestClassifier(max_depth=16)

# fit the model 
forest.fit(X_train, y_train)

In [ ]:
#predicting the target value from the model for the samples
y_test_forest = forest.predict(X_test)
y_train_forest = forest.predict(X_train)

#### Performance Evaluation

In [ ]:

#computing the accuracy of the model performance
acc_train_forest = accuracy_score(y_train,y_train_forest)
acc_test_forest = accuracy_score(y_test,y_test_forest)

print("Random forest: Accuracy on training Data: {:.3f}".format(acc_train_forest))
print("Random forest: Accuracy on test Data: {:.3f}".format(acc_test_forest))

In [ ]:
#checking the feature improtance in the model
plt.figure(figsize=(9,7))
n_features = X_train.shape[1]
plt.barh(range(n_features), forest.feature_importances_, align='center')
plt.yticks(np.arange(n_features), X_train.columns)
plt.xlabel("Feature importance")
plt.ylabel("Feature")
plt.show()

#### Storing Results

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('Random Forest', acc_train_forest, acc_test_forest)

### 10.3. XGBoost

XGBoost is one of the most popular machine learning algorithms these days. XGBoost stands for eXtreme Gradient Boosting. Regardless of the type of prediction task at hand; regression or classification. XGBoost is an implementation of gradient boosted decision trees designed for speed and performance.

In [ ]:
#XGBoost Classification model
from xgboost import XGBClassifier

# instantiate the model
xgb = XGBClassifier(learning_rate=0.4,max_depth=7)
#fit the model
xgb.fit(X_train, y_train)

In [ ]:
#predicting the target value from the model for the samples
y_test_xgb = xgb.predict(X_test)
y_train_xgb = xgb.predict(X_train)

#### Performance Evaluation

In [ ]:
#computing the accuracy of the model performance
acc_train_xgb = accuracy_score(y_train,y_train_xgb)
acc_test_xgb = accuracy_score(y_test,y_test_xgb)

print("XGBoost: Accuracy on training Data: {:.3f}".format(acc_train_xgb))
print("XGBoost : Accuracy on test Data: {:.3f}".format(acc_test_xgb))

#### Storing Results

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('XGBoost', acc_train_xgb, acc_test_xgb)

### 10.4. Logistic Regression

In machine learning, logistic regression is a classification algorithm used to model the relationship between input features and a binary output variable. It estimates the probability of the output belonging to a specific class using a logistic or sigmoid function.
During training, it learns optimal parameters by minimizing a cost function, such as cross-entropy loss. Once trained, it can make predictions on new data by calculating the probability of belonging to a class. Logistic regression is widely used in various domains for binary classification tasks when a linear relationship is assumed.

In [ ]:
#Logistic regression machine learning

from sklearn.linear_model import LogisticRegression

#instantive the model
logistic = LogisticRegression(random_state=12, max_iter=1000)
#fit the model
logistic.fit(X_train, y_train)

In [ ]:
#predicting the target value from the model for the samples
y_test_log = logistic.predict(X_test)
y_train_log = logistic.predict(X_train)

#### Performance Evaluation

In [ ]:
#computing the accuracy of the model performance
acc_train_log = accuracy_score(y_train,y_train_log)
acc_test_log = accuracy_score(y_test,y_test_log)

print("Logistic: Accuracy on training Data: {:.3f}".format(acc_train_log))
print("Logistic : Accuracy on test Data: {:.3f}".format(acc_test_log))

#### Storing Result

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('Logistic', acc_train_log, acc_test_log)

### 10.5. Support Vector Machines

In machine learning, support-vector machines (SVMs, also support-vector networks) are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis. Given a set of training examples, each marked as belonging to one or the other of two categories, an SVM training algorithm builds a model that assigns new examples to one category or the other, making it a non-probabilistic binary linear classifier.

In [ ]:
#Support vector machine model
from sklearn.svm import SVC

# instantiate the model
svm = SVC(kernel='linear', C=1.0, random_state=12)
#fit the model
svm.fit(X_train, y_train)

In [ ]:
#predicting the target value from the model for the samples
y_test_svm = svm.predict(X_test)
y_train_svm = svm.predict(X_train)

#### Performance Evaluation

In [ ]:
#computing the accuracy of the model performance
acc_train_svm = accuracy_score(y_train,y_train_svm)
acc_test_svm = accuracy_score(y_test,y_test_svm)

print("SVM: Accuracy on training Data: {:.3f}".format(acc_train_svm))
print("SVM : Accuracy on test Data: {:.3f}".format(acc_test_svm))

#### Storing Result

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('SVM', acc_train_svm, acc_test_svm)

### 10.6. Multilayer Perceptrons (MLPs)

Multilayer perceptrons (MLPs) are also known as (vanilla) feed-forward neural networks, or sometimes just neural networks. Multilayer perceptrons can be applied for both classification and regression problems.

MLPs can be viewed as generalizations of linear models that perform multiple stages of processing to come to a decision.

In [ ]:
# Multilayer Perceptrons model
from sklearn.neural_network import MLPClassifier

# instantiate the model
mlp = MLPClassifier(alpha=0.001, hidden_layer_sizes=([100,100,100]))

# fit the model 
mlp.fit(X_train, y_train)

In [ ]:
#predicting the target value from the model for the samples
y_test_mlp = mlp.predict(X_test)
y_train_mlp = mlp.predict(X_train)

#### Performance Evaluation

In [ ]:
#computing the accuracy of the model performance
acc_train_mlp = accuracy_score(y_train,y_train_mlp)
acc_test_mlp = accuracy_score(y_test,y_test_mlp)

print("Multilayer Perceptrons: Accuracy on training Data: {:.3f}".format(acc_train_mlp))
print("Multilayer Perceptrons: Accuracy on test Data: {:.3f}".format(acc_test_mlp))

#### Storing Result

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('Multilayer Perceptrons', acc_train_mlp, acc_test_mlp)

## 11. Perfect Score - Dropping Features

After consulation with our mentor Amit Shtekel and explaining him we received a perfect score for our models (1.00) , he gave us new assignment , and it is to compare the price of extracting each feature. After we will drop the expensive features we will need to check the model again and to see it's results.

We decided to measure the run time of the features after we grouped them to categories , parsing , domain and dns registery, content.
After a few runtime measures we have seen that the extraction of the domain and dns registeries are much more expensive - for 1000 rows the parsing features  took 0.09 sec and the content features took 0.9 seconds , but the domain/dns features took 620 seconds(!!) , compared to the other features combined it is more than 620 times .

## 12. Dropping the domain/dns features 

## 12.1 Loading the Data

In [ ]:
urldata = pd.read_csv("/kaggle/working/urldata.csv")
urldata.head()

## 12.2 Dropping Features

In [ ]:
urldata = urldata.drop(["dns"], axis=1)
urldata = urldata.drop(["domainAge"], axis=1)
urldata = urldata.drop(["domainEnd"], axis=1)
urldata = urldata.drop(["iframe"], axis=1)
urldata = urldata.drop(["forwarding"], axis=1)
urldata.head()

## 13.Training Models

## 13.1 Splitting the Data

In [ ]:
y = urldata['label']
X = urldata.drop('label',axis=1)
X.shape, y.shape

In [ ]:
# Splitting the dataset into train and test sets: 70-30 split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.3, random_state = 42)
X_train.shape, X_test.shape

## Random Forest

In [ ]:
# Random Forest model
from sklearn.ensemble import RandomForestClassifier

# instantiate the model
forest = RandomForestClassifier(max_depth=5)

# fit the model 
forest.fit(X_train, y_train)

In [ ]:
#predicting the target value from the model for the samples
y_test_forest = forest.predict(X_test)
y_train_forest = forest.predict(X_train)

In [ ]:

#computing the accuracy of the model performance
acc_train_forest = accuracy_score(y_train,y_train_forest)
acc_test_forest = accuracy_score(y_test,y_test_forest)

print("Random forest: Accuracy on training Data: {:.3f}".format(acc_train_forest))
print("Random forest: Accuracy on test Data: {:.3f}".format(acc_test_forest))

## Logistic Regression

In [ ]:
#Logistic regression machine learning

from sklearn.linear_model import LogisticRegression

#instantive the model
logistic = LogisticRegression(random_state=42, max_iter=1000)
#fit the model
logistic.fit(X_train, y_train)

In [ ]:
#predicting the target value from the model for the samples
y_test_ogisticl = logistic.predict(X_test)
y_train_logistic = logistic.predict(X_train)

In [ ]:

#computing the accuracy of the model performance
acc_train_logistic = accuracy_score(y_train,y_train_logistic)
acc_test_logistic = accuracy_score(y_test,y_test_ogisticl)

print("Logistic Regression: Accuracy on training Data: {:.3f}".format(acc_train_logistic))
print("Logistic Regression: Accuracy on test Data: {:.3f}".format(acc_test_logistic))